In [8]:
import pandas as pd
import numpy as np
import time
import fpGrowth 

In [2]:
#预处理数据
def pre_data(df):
    df['timestamp'] = pd.to_datetime(df['sldat'])
    
    #分别按vipno、sldat排序
    data = df.sort_values(by=['vipno', 'sldat'])
    
    #分组
    data['rank'] = data['timestamp'].groupby(data['vipno']).rank(ascending=0, method='first')
    
    #取每个vipno的前60%的数据
    data = data.groupby(['vipno'], as_index = False).apply(lambda x: x[x['rank'] <=  round(0.6 * x['rank'].max())])
    #整理
    data = data.drop(['timestamp', 'rank'], axis = 1).reset_index(drop=True)
    return data

In [3]:

#提出dptno的数据
def pre_dptno(data):
    #丢弃多余的列
    data_dptno = data.drop(['sldat', 'vipno', 'pluno', 'bndno'], axis = 1)
    
    #合并订单
    data_dptno['value'] = data_dptno['dptno']
    data_dptno = data_dptno.pivot_table(data_dptno , index = ['uid'], columns = 'dptno')
    
    #整理
    data_dptno = data_dptno.fillna(0).transpose().loc['value'].transpose()
    del data_dptno.index.name
    del data_dptno.transpose().index.name
    
    #将dataframe转为array
    array_dptno = []
    for row in data_dptno.as_matrix():
        array_dptno.append([x for x in row if x != 0.0])
    return array_dptno

In [6]:

def fp_dptno(data):
    thresholds = [2, 4, 8, 16, 32, 64]
    array_dptno = pre_dptno(data)
    for n in thresholds:
        freq_sets = fpGrowth.fpGrowth(array_dptno, n)
        print("for dptno, threshold: ", n)
        for k in freq_sets:
            if len(k) >= 2:
                print (k, freq_sets[k])

In [9]:
if __name__ == "__main__":
    old_df = pd.read_csv('../trade.csv', usecols=['uid', 'vipno', 'sldat', 'pluno', 'dptno', 'bndno'])
    #旧数据
    old_data = pre_data(old_df)
    start = time.clock()
    fp_dptno(old_data)
    time_old = time.clock() - start
    #新数据
    new_df = pd.read_csv('../trade_new.csv', usecols=['uid', 'vipno', 'sldatime', 'pluno', 'dptno', 'bndno'])
    new_df.rename(columns={ new_df.columns[1]: "sldat" }, inplace=True)    
    new_data = pre_data(new_df)
    start = time.clock()
    fp_dptno(new_data)
    time_new = time.clock() - start
        
    print("fp_dptno,for old data: ", time_old)
    print("fp_dptno,for new data: ", time_new)

for dptno, threshold:  2
frozenset({30380.0, 30500.0}) 2
frozenset({14661.0, 14654.0}) 2
frozenset({27000.0, 14230.0}) 2
frozenset({14082.0, 14083.0}) 2
frozenset({22008.0, 10100.0}) 2
frozenset({10310.0, 15110.0}) 2
frozenset({23110.0, 10310.0}) 2
frozenset({23110.0, 10310.0, 15110.0}) 2
frozenset({11522.0, 25111.0}) 2
frozenset({27000.0, 11522.0}) 2
frozenset({27000.0, 11522.0, 25111.0}) 2
frozenset({25120.0, 14772.0}) 2
frozenset({14753.0, 15110.0}) 2
frozenset({30380.0, 14021.0}) 2
frozenset({30380.0, 14141.0}) 2
frozenset({30304.0, 30380.0}) 2
frozenset({30304.0, 14092.0}) 2
frozenset({14130.0, 14775.0}) 2
frozenset({14091.0, 14031.0}) 2
frozenset({14870.0, 15119.0}) 2
frozenset({15001.0, 23110.0}) 2
frozenset({30800.0, 30380.0}) 2
frozenset({15115.0, 14140.0}) 2
frozenset({15130.0, 14140.0}) 2
frozenset({15130.0, 15115.0, 14140.0}) 2
frozenset({15211.0, 10119.0}) 2
frozenset({15211.0, 30380.0}) 2
frozenset({15211.0, 30380.0, 10119.0}) 2
frozenset({22600.0, 15234.0}) 2
frozenset({

frozenset({23113.0, 15113.0, 24010.0, 15502.0}) 2
frozenset({24010.0, 10150.0, 15502.0}) 2
frozenset({23113.0, 24010.0, 10150.0, 15502.0}) 2
frozenset({15113.0, 24010.0, 10150.0, 15502.0}) 2
frozenset({10150.0, 15113.0, 23113.0, 24010.0, 15502.0}) 2
frozenset({14802.0, 15502.0}) 2
frozenset({22008.0, 15502.0}) 2
frozenset({22008.0, 10150.0, 15502.0}) 2
frozenset({22008.0, 15113.0, 15502.0}) 2
frozenset({22008.0, 24010.0, 15502.0}) 2
frozenset({22008.0, 15113.0, 24010.0, 15502.0}) 2
frozenset({22008.0, 23113.0, 15502.0}) 2
frozenset({10300.0, 15502.0}) 2
frozenset({10300.0, 15502.0, 10150.0}) 2
frozenset({15113.0, 10300.0, 15502.0}) 2
frozenset({15113.0, 10300.0, 15502.0, 10150.0}) 2
frozenset({24010.0, 10300.0, 15502.0}) 2
frozenset({15113.0, 24010.0, 10300.0, 15502.0}) 2
frozenset({24010.0, 10300.0, 15502.0, 10150.0}) 2
frozenset({10150.0, 15113.0, 24010.0, 10300.0, 15502.0}) 2
frozenset({23113.0, 10300.0, 15502.0}) 2
frozenset({23113.0, 10300.0, 15113.0, 15502.0}) 2
frozenset({23113.

frozenset({14082.0, 22111.0}) 2
frozenset({14082.0, 27300.0, 22111.0}) 2
frozenset({14082.0, 22103.0, 22111.0}) 2
frozenset({14082.0, 27300.0, 22103.0, 22111.0}) 2
frozenset({14082.0, 23132.0, 22111.0}) 2
frozenset({14082.0, 15004.0}) 2
frozenset({14082.0, 25101.0}) 3
frozenset({14082.0, 23110.0}) 3
frozenset({14082.0, 30380.0, 23110.0}) 2
frozenset({14082.0, 15119.0}) 3
frozenset({14082.0, 23132.0, 15119.0}) 2
frozenset({14082.0, 23132.0, 27300.0, 15119.0}) 2
frozenset({14082.0, 24101.0, 15119.0}) 2
frozenset({14082.0, 27300.0, 24101.0, 15119.0}) 2
frozenset({14082.0, 27300.0, 15119.0}) 3
frozenset({27400.0, 14082.0}) 3
frozenset({14082.0, 23134.0}) 3
frozenset({27400.0, 14082.0, 23134.0}) 2
frozenset({14082.0, 27002.0}) 4
frozenset({14082.0, 23132.0, 27002.0}) 2
frozenset({14082.0, 23132.0, 27300.0, 27002.0}) 2
frozenset({14082.0, 27300.0, 27002.0}) 3
frozenset({14082.0, 22103.0}) 4
frozenset({14082.0, 27300.0, 22103.0}) 3
frozenset({14082.0, 23132.0, 27300.0, 22103.0}) 3
frozenset({

frozenset({27410.0, 23132.0}) 8
frozenset({27410.0, 23132.0, 15110.0}) 2
frozenset({27410.0, 23132.0, 27300.0}) 4
frozenset({23132.0, 15110.0}) 9
frozenset({23110.0, 23132.0, 15110.0}) 3
frozenset({23132.0, 22103.0}) 9
frozenset({27300.0, 23132.0, 22103.0}) 3
frozenset({27300.0, 23132.0}) 10
frozenset({23132.0, 23110.0}) 11
frozenset({27200.0, 22103.0}) 4
frozenset({27200.0, 25101.0, 22103.0}) 2
frozenset({27200.0, 27000.0, 22103.0}) 2
frozenset({27002.0, 22103.0}) 7
frozenset({25120.0, 27002.0, 22103.0}) 2
frozenset({25120.0, 27002.0, 27000.0, 22103.0}) 2
frozenset({27002.0, 22102.0, 22103.0}) 2
frozenset({27002.0, 27300.0, 22102.0, 22103.0}) 2
frozenset({27002.0, 27410.0, 22103.0}) 2
frozenset({27000.0, 27002.0, 27410.0, 22103.0}) 2
frozenset({27002.0, 25101.0, 22103.0}) 2
frozenset({27000.0, 27002.0, 25101.0, 22103.0}) 2
frozenset({27000.0, 27002.0, 22103.0}) 3
frozenset({27002.0, 27300.0, 22103.0}) 3
frozenset({27410.0, 22103.0}) 7
frozenset({27410.0, 27300.0, 22103.0}) 2
frozenset

frozenset({30380.0, 25101.0, 22102.0}) 4
frozenset({27000.0, 22102.0}) 11
frozenset({27000.0, 27300.0, 22102.0}) 4
frozenset({23110.0, 22102.0}) 14
frozenset({27300.0, 22102.0}) 16
frozenset({30380.0, 27300.0, 22102.0}) 5
frozenset({30380.0, 22102.0}) 20
frozenset({25101.0, 15110.0}) 4
frozenset({15114.0, 25101.0}) 4
frozenset({27002.0, 25101.0}) 11
frozenset({27002.0, 27410.0, 25101.0}) 4
frozenset({27000.0, 27002.0, 25101.0}) 4
frozenset({25101.0, 23110.0}) 12
frozenset({27300.0, 25101.0}) 12
frozenset({27410.0, 27300.0, 25101.0}) 5
frozenset({30380.0, 27300.0, 25101.0}) 5
frozenset({30380.0, 25101.0}) 14
frozenset({27410.0, 30380.0, 25101.0}) 4
frozenset({27000.0, 30380.0, 25101.0}) 5
frozenset({27000.0, 25101.0}) 15
frozenset({27000.0, 27410.0, 25101.0}) 7
frozenset({27410.0, 25101.0}) 17
frozenset({15114.0, 27002.0}) 4
frozenset({27000.0, 15114.0}) 7
frozenset({27000.0, 15114.0, 27410.0}) 4
frozenset({15114.0, 27410.0}) 8
frozenset({15114.0, 15130.0}) 10
frozenset({15114.0, 15110.

...

frozenset({27200.0, 27300.0, 27000.0}) 5
frozenset({27200.0, 30380.0, 27300.0, 27000.0}) 3
frozenset({27200.0, 30380.0, 27300.0}) 6
frozenset({27200.0, 23110.0}) 14
frozenset({27200.0, 27300.0, 23110.0}) 2
frozenset({27200.0, 27000.0, 23110.0}) 3
frozenset({27200.0, 27410.0, 23110.0}) 5
frozenset({27200.0, 27410.0, 30380.0, 23110.0}) 3
frozenset({27200.0, 30380.0, 23110.0}) 6
frozenset({27200.0, 27410.0}) 18
frozenset({27200.0, 27410.0, 27000.0}) 4
frozenset({27200.0, 30380.0, 27410.0, 27000.0}) 4
frozenset({27200.0, 27410.0, 30380.0}) 10
frozenset({27200.0, 27000.0}) 19
frozenset({27200.0, 30380.0, 27000.0}) 9
frozenset({27200.0, 30380.0}) 28
frozenset({15200.0, 10141.0}) 2
frozenset({15200.0, 23132.0}) 3
frozenset({15200.0, 23132.0, 15110.0}) 2
frozenset({15200.0, 22008.0}) 3
frozenset({15200.0, 22008.0, 15110.0}) 3
frozenset({15200.0, 24010.0}) 4
frozenset({15200.0, 24010.0, 22111.0}) 2
frozenset({15200.0, 24010.0, 30380.0, 22111.0}) 2
frozenset({15200.0, 24010.0, 22102.0, 22111.0})

frozenset({27410.0, 15115.0, 15110.0}) 2
frozenset({22008.0, 27410.0, 15115.0, 15110.0}) 2
frozenset({27410.0, 15115.0, 15110.0, 15130.0}) 2
frozenset({27410.0, 15110.0, 22008.0, 15130.0, 15115.0}) 2
frozenset({27410.0, 15115.0, 15130.0}) 3
frozenset({27410.0, 15115.0, 30380.0, 15130.0}) 2
frozenset({27410.0, 22008.0, 15130.0, 15115.0, 30380.0}) 2
frozenset({22008.0, 27410.0, 15115.0, 15130.0}) 3
frozenset({27410.0, 15115.0, 30380.0}) 6
frozenset({22008.0, 27410.0, 15115.0, 30380.0}) 6
frozenset({22008.0, 27410.0, 15115.0}) 7
frozenset({22008.0, 15115.0}) 12
frozenset({22008.0, 15115.0, 15110.0}) 2
frozenset({22008.0, 15130.0, 15115.0, 15110.0}) 2
frozenset({22008.0, 15130.0, 15115.0}) 3
frozenset({22008.0, 15130.0, 15115.0, 30380.0}) 2
frozenset({22008.0, 15115.0, 30380.0}) 11
frozenset({15130.0, 15115.0}) 14
frozenset({15130.0, 15115.0, 23110.0}) 3
frozenset({15130.0, 15115.0, 30380.0, 23110.0}) 2
frozenset({15130.0, 15115.0, 15110.0}) 4
frozenset({15130.0, 15115.0, 30380.0, 15110.0}

frozenset({27410.0, 30380.0, 15231.0}) 4
frozenset({15110.0, 15231.0}) 9
frozenset({30380.0, 15110.0, 15231.0}) 4
frozenset({23110.0, 15231.0}) 9
frozenset({15130.0, 23110.0, 15231.0}) 4
frozenset({15130.0, 15231.0}) 15
frozenset({15130.0, 30380.0, 15231.0}) 7
frozenset({30380.0, 15231.0}) 21
frozenset({24010.0, 14802.0}) 4
frozenset({14802.0, 15115.0}) 4
frozenset({22008.0, 14802.0}) 4
frozenset({14802.0, 10141.0}) 5
frozenset({14802.0, 15110.0}) 6
frozenset({14802.0, 27410.0}) 7
frozenset({14802.0, 23110.0}) 8
frozenset({14802.0, 30380.0, 23110.0}) 4
frozenset({14802.0, 30380.0}) 17
frozenset({22008.0, 10150.0}) 4
frozenset({22008.0, 30380.0, 10150.0}) 4
frozenset({10000.0, 10150.0}) 4
frozenset({27410.0, 10150.0}) 4
frozenset({27000.0, 10150.0}) 4
frozenset({10150.0, 10119.0}) 5
frozenset({30380.0, 10150.0, 10119.0}) 4
frozenset({10141.0, 10150.0}) 5
frozenset({14092.0, 10150.0}) 5
frozenset({14091.0, 10150.0}) 5
frozenset({15202.0, 10150.0}) 6
frozenset({15202.0, 30380.0, 10150.0})

frozenset({24010.0, 27300.0}) 11
frozenset({30380.0, 24010.0, 27300.0}) 4
frozenset({24010.0, 22111.0}) 13
frozenset({24010.0, 22102.0, 22111.0}) 7
frozenset({24010.0, 30380.0, 22102.0, 22111.0}) 4
frozenset({24010.0, 30380.0, 22111.0}) 8
frozenset({24010.0, 27410.0}) 13
frozenset({24010.0, 27410.0, 22102.0}) 4
frozenset({24010.0, 27410.0, 30380.0}) 6
frozenset({24010.0, 15110.0}) 14
frozenset({24010.0, 22102.0}) 18
frozenset({24010.0, 30380.0, 22102.0}) 10
frozenset({24010.0, 30380.0}) 36
frozenset({27000.0, 15114.0}) 6
frozenset({27000.0, 15114.0, 30380.0}) 6
frozenset({22008.0, 15114.0}) 7
frozenset({22008.0, 15114.0, 30380.0}) 4
frozenset({15114.0, 22102.0}) 7
frozenset({15114.0, 27002.0}) 11
frozenset({15114.0, 27410.0, 27002.0}) 4
frozenset({15114.0, 30380.0, 27002.0}) 8
frozenset({15114.0, 27410.0}) 15
frozenset({15114.0, 27410.0, 23110.0}) 4
frozenset({15114.0, 27410.0, 15110.0}) 4
frozenset({15114.0, 27410.0, 30380.0}) 8
frozenset({15114.0, 15130.0}) 15
frozenset({15114.0, 151

frozenset({15115.0, 14091.0}) 10
frozenset({15130.0, 14091.0}) 10
frozenset({14091.0, 15110.0}) 11
frozenset({14091.0, 14092.0}) 30
frozenset({30380.0, 14091.0, 14092.0}) 16
frozenset({14091.0, 30380.0}) 33
frozenset({14092.0, 10141.0}) 9
frozenset({27410.0, 14092.0}) 9
frozenset({14092.0, 15110.0}) 9
frozenset({15115.0, 14092.0}) 10
frozenset({14092.0, 23110.0}) 11
frozenset({15130.0, 14092.0}) 11
frozenset({30380.0, 14092.0}) 37
frozenset({27400.0, 25120.0}) 8
frozenset({27400.0, 10141.0}) 8
frozenset({27400.0, 15114.0}) 8
frozenset({27400.0, 22102.0}) 9
frozenset({27400.0, 27002.0}) 11
frozenset({27400.0, 27000.0}) 12
frozenset({27400.0, 27410.0}) 13
frozenset({27400.0, 23110.0}) 14
frozenset({27400.0, 15130.0}) 15
frozenset({27400.0, 30380.0}) 26
frozenset({22008.0, 15202.0}) 8
frozenset({15202.0, 27410.0}) 9
frozenset({15202.0, 15110.0}) 9
frozenset({15202.0, 15114.0}) 11
frozenset({15202.0, 15130.0}) 12
frozenset({15202.0, 23110.0}) 14
frozenset({15202.0, 30380.0}) 25
frozenset({